## Method1 - LinkedList


In [2]:
from collections import defaultdict

class Node:
    def __init__(self, key, value):
        self.key = key
        self.value = value
        self.freq = 1  # Initialize frequency to 1
        self.prev = self.next = None

class DoublyLinkedList:
    def __init__(self):
        self.head = Node(None, None)
        self.tail = Node(None, None)
        self.head.next, self.tail.prev = self.tail, self.head
        self.size = 0
    
    def insert(self, node):
        node.next, node.prev = self.head.next, self.head
        self.head.next.prev, self.head.next = node, node
        self.size += 1
    
    def remove(self, node):
        node.prev.next, node.next.prev = node.next, node.prev
        self.size -= 1
    
    def pop(self):
        if self.size > 0:
            node = self.tail.prev
            self.remove(node)
            return node
        return None  # Return None if list is empty

class LFUCache:

    def __init__(self, capacity: int):
        self.capacity = capacity
        self.cache = {}  # Map key to node
        self.freq_map = defaultdict(DoublyLinkedList)  # Map frequency to DLL of nodes
        self.min_freq = 0

    def get(self, key: int) -> int:
        if key not in self.cache:
            return -1
        
        node = self.cache[key]
        self._update(node)
        return node.value
    
    def put(self, key: int, value: int) -> None:
        if self.capacity == 0:
            return
        
        if key in self.cache:
            node = self.cache[key]
            node.value = value
            self._update(node)
        else:
            if len(self.cache) >= self.capacity:
                min_freq_list = self.freq_map[self.min_freq]
                node_to_remove = min_freq_list.pop()  # Pop the LRU node from the least frequent list
                if node_to_remove:  # Make sure it's not None
                    del self.cache[node_to_remove.key]
            
            new_node = Node(key, value)
            self.cache[key] = new_node
            self.freq_map[1].insert(new_node)
            self.min_freq = 1  # Reset the min frequency to 1

    def _update(self, node):
        freq = node.freq
        self.freq_map[freq].remove(node)
        
        if freq == self.min_freq and self.freq_map[freq].size == 0:
            self.min_freq += 1
        
        node.freq += 1
        self.freq_map[node.freq].insert(node)

# Test case
lfu_cache = LFUCache(2)
lfu_cache.put(1, 1)
lfu_cache.put(2, 2)
print(lfu_cache.get(1))  # Expected output: 1
lfu_cache.put(3, 3)  # Evicts key 2
print(lfu_cache.get(2))  # Expected output: -1 (not found)
print(lfu_cache.get(3))  # Expected output: 3
lfu_cache.put(4, 4)  # Evicts key 1
print(lfu_cache.get(1))  # Expected output: -1 (not found)
print(lfu_cache.get(3))  # Expected output: 3
print(lfu_cache.get(4))  # Expected output: 4


1
-1
3
-1
3
4
